In [6]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
import sys
sys.path.append("/home/resperanca/Tuberculosis_Tese/Source/Models/BarlowDTI")

In [ ]:
sys.path.append("/home/resperanca/Tuberculosis_Tese/Source/Models/BarlowDTI/utils")

from utils.sequence import uniprot2sequence, encode_sequences
from utils.chem import *
from utils.parallel import *
from utils.sequence import encode_sequences
from utils.chem import get_mols, get_fp  

In [8]:
import pandas as pd
import numpy as np
import pickle
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score, average_precision_score
from rdkit import Chem
from xgboost import XGBClassifier

sys.path.append("/home/resperanca/Tuberculosis_Tese/Source/Models/BarlowDTI/model")

from base_model import BaseModel
from preprocessor import Preprocessor
from barlow_twins import BarlowTwins


### Model Run

In [9]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, recall_score,
    f1_score, matthews_corrcoef, confusion_matrix, average_precision_score
)
import seaborn as sns
import matplotlib.pyplot as plt

test_path = "/home/resperanca/Tuberculosis_Tese/Data/DTI_Data_2_step/all_train_val_tb_papyrus.csv"  
barlow_model_path = "/home/resperanca/Tuberculosis_Tese/Source/Models/BarlowDTI/model/stash/Papyrus_tb"  

bt_model = BarlowTwins()
bt_model.load_model(barlow_model_path)


test_df = pd.read_csv(test_path)

# ECFPs e embeddings 
test_mols = [Chem.MolFromSmiles(smi) for smi in test_df["smiles"]]
test_ecfp = [AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=1024) for m in test_mols]
test_ecfp = np.array(test_ecfp)

test_emb = encode_sequences(test_df["sequence"].tolist(), encoder="prost_t5")
test_emb = np.array([np.array(x) for x in test_emb])

# Conct embeddings
test_vectors = bt_model.zero_shot(test_ecfp, test_emb)
true_labels = test_df["label"].values

# === 5. Carregar modelo treinado ===
#model = XGBClassifier()
#model.load_model("/home/resperanca/Tuberculosis_Tese/Source/Models/BarlowDTI/model/xxl_stash/Papyrus_tb_barlowdti_xxl_model_tb_pap.json")




[BT]: Model constructed successfully
[BT]: Optimizer constructed successfully
[BT]: Current parameter config:
{'act_function': <class 'torch.nn.modules.activation.ReLU'>, 'loss_weight': 0.005, 'batch_size': 512, 'learning_rate': 0.0001, 'betas': (0.9, 0.999), 'step_size': 10, 'gamma': 0.1, 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'momentum': 0.9, 'enc_n_neurons': 512, 'enc_n_layers': 2, 'proj_n_neurons': 2048, 'proj_n_layers': 2, 'embedding_dim': 512, 'weight_decay': 0.001, 'verbose': True, 'radius': 'Not defined yet', 'n_bits': 1024, 'aa_emb_size': 1024}


/home/resperanca/Tuberculosis_Tese/Source/Models/BarlowDTI/model/barlow_twins.py:491: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(weights_path, map

[BT]: Model constructed successfully
[BT]: Optimizer constructed successfully
[BT]: Model loaded from /home/resperanca/Tuberculosis_Tese/Source/Models/BarlowDTI/model/stash/Papyrus_tb
[BT]: Loaded parameters:
{'act_function': <class 'torch.nn.modules.activation.ReLU'>, 'loss_weight': 0.001, 'batch_size': 512, 'learning_rate': 0.0001, 'betas': (0.9, 0.999), 'step_size': 10, 'gamma': 0.1, 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'momentum': 0.9, 'enc_n_neurons': 4096, 'enc_n_layers': 3, 'proj_n_neurons': 4096, 'proj_n_layers': 1, 'embedding_dim': 256, 'weight_decay': 0.0015, 'verbose': True, 'radius': 2, 'n_bits': 1024, 'aa_emb_size': 1024}


Encoding sequences: 100%|██████████| 451272/451272 [07:34<00:00, 993.65it/s]  


## SHAP (top 20 bit)

In [ ]:

import shap

#explainer = shap.Explainer(model, test_vectors)
#shap_values = explainer(test_vectors)

# === 7. Isolar os bits ECFP (assumindo que vêm antes no vetor) ===
# Se ECFP tem 1024 bits:
#ecfp_bits_only = test_vectors[:, :1024]
#shap_ecfp_values = shap_values.values[:, :1024]

# === 8. Calcular importância média por bit ===
#mean_abs_shap = np.abs(shap_ecfp_values).mean(axis=0)
#top_bits_idx = np.argsort(mean_abs_shap)[::-1][:20]

#print("\nTop 20 bits ECFP mais importantes:")
#for i in top_bits_idx:
    #print(f"Bit {i}: SHAP value média = {mean_abs_shap[i]:.4f}")


100%|===================| 451250/451272 [276:36<00:00]         


Top 20 bits ECFP mais importantes:
Bit 49: SHAP value média = 0.1621
Bit 472: SHAP value média = 0.1350
Bit 178: SHAP value média = 0.0892
Bit 123: SHAP value média = 0.0811
Bit 100: SHAP value média = 0.0788
Bit 0: SHAP value média = 0.0697
Bit 137: SHAP value média = 0.0662
Bit 223: SHAP value média = 0.0629
Bit 39: SHAP value média = 0.0597
Bit 88: SHAP value média = 0.0595
Bit 156: SHAP value média = 0.0594
Bit 91: SHAP value média = 0.0591
Bit 106: SHAP value média = 0.0576
Bit 72: SHAP value média = 0.0570
Bit 71: SHAP value média = 0.0567
Bit 160: SHAP value média = 0.0544
Bit 218: SHAP value média = 0.0541
Bit 61: SHAP value média = 0.0538
Bit 6: SHAP value média = 0.0535
Bit 58: SHAP value média = 0.0534


# Top 50 molecules whit more bits

In [ ]:
import numpy as np 

important_bits = top_bits_idx 


bit_presence_matrix = np.array([
    [int(fp[i]) for i in important_bits] for fp in test_ecfp
])

bit_activation_score = bit_presence_matrix.sum(axis=1)
top_mol_indices = np.argsort(bit_activation_score)[::-1][:50]

print("Top 50 moléculas com mais bits importantes ativados:")
for idx in top_mol_indices:
    print(f"Índice {idx}, SMILES: {test_df.iloc[idx]['smiles']}, Ativa: {test_df.iloc[idx]['label']}, Score: {bit_activation_score[idx]}")


for idx in top_mol_indices:
    print(f"{test_df.iloc[idx]['smiles']}")


Top 50 moléculas com mais bits importantes ativados:
Índice 389577, SMILES: N#Cc1cnc(NC(=O)N2CCCc3ccc(C(=O)C(F)(F)F)nc32)cc1OC1CCCC1, Ativa: 1, Score: 5
Índice 277461, SMILES: CN(C)CC(C)(C)COc1cc(NC(=O)N2CCCc3cc(C(F)F)c(C=O)nc32)ncc1C#N, Ativa: 0, Score: 5
Índice 124616, SMILES: C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CC2OC(=O)C(=C)C2CCCC)CC(O)C(OCCCO)C1O, Ativa: 1, Score: 5
Índice 404932, SMILES: Cc1cc2c(C(C)C)c(O)c(O)c(C=O)c2c(O)c1-c1cc2cc(C(C)C)c(O)c(O)c2c(C=O)c1O, Ativa: 0, Score: 5
Índice 425993, SMILES: C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CC2OC(=O)C(=C)C2CC)CC(O)C(OCCCO)C1O, Ativa: 1, Score: 5
Índice 120371, SMILES: CN(C)CC(C)(C)COc1cc(NC(=O)N2CCCc3cc(C(F)F)c(C=O)nc32)ncc1C#N, Ativa: 1, Score: 5
Índice 270410, SMILES: N#Cc1cnc(NC(=O)N2CCCc3ccc(C(=O)C(F)(F)F)nc32)cc1OC1CCCCC1, Ativa: 1, Score: 5
Índice 261123, SMILES: C=C1C(=O)OC2CC3(C)CCC=C(COC(=O)CC4CCN(C(=O)c5cc(Cc6n[nH]c(=O)c7ccccc67)ccc5F)CC4)C3=CC12, Ativa: 1, Score: 5
Índice 393569, SMILES: Cc1cc2c(C(C)C)c(O)c(O)c(C=O)c2c(O)c1-c1cc2cc(C(C)

# SHAP VS RESULTS (PRESENCE BITS)

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np


smiles_list_14 = ['CNC(=O)C(Cc1ccccc1)NC(=O)C(CC(C)C)C1(C(=O)NO)CCCC1',
'CNC(=O)C(Cc1ccccc1)NC(=O)C(CC(C)C)C1(C(=O)NO)CCCC1',
'COc1cc(C#N)cc(-c2csc(C34CN(c5ncc(F)cn5)CC3C(=O)N(C)C(N)=N4)c2)c1',
'Cc1c(CC(=O)O)cc2ccc(F)cc2c1-c1ccc(S(=O)(=O)N2CCCC2)cc1',
'O=C(NCC1CN(c2ccc(N3CCCC(O)C3)c(F)c2)C(=O)O1)c1ccc(Cl)s1',
'CCc1coc(NC(=O)c2cc3c(nc2OC)nc(C(O)(c2ccccc2)c2ccccc2)n3CC)n1',
'CNC(=O)C1(c2cccc(OCc3cc(C)nc4ccccc34)c2)CC1C(=O)NO',
'Cn1cnc(S(=O)(=O)Nc2cccc(-c3ccc(C(=O)c4cc(F)c(F)c(O)c4F)s3)c2)c1',
'Nc1cncc(C2=NN(C(=O)c3ccc(-c4ccccn4)s3)C(c3ccccc3O)C2)c1',
'O=C(NCC1CN(c2ccc(N3CCC(O)C3)c(F)c2)C(=O)O1)c1ccc(Cl)s1',
'O=C(NCC1CN(c2ccc(N3CCC(O)C3)c(F)c2)C(=O)O1)c1ccc(Cl)s1',
'O=C(NCC1CN(c2ccc(N3CCC(O)C3)c(F)c2)C(=O)O1)c1ccc(Cl)s1',
'NC(=O)c1cc(-c2ccc(Cl)c(Cl)c2)cc2c1[nH]c1ccc(C(=O)N3CCOC(CO)C3)cc12',
'N#Cc1ccc(NC(=O)c2ccc(N3CCCCC3=O)cc2)c(C(=O)Nc2ccc(Cl)cn2)c1']

smiles_list_1 = ['CCCS(=O)(=O)C1CCN(c2ccn3cc(-c4cc(Cl)c(OC)cc4OC)nc3n2)CC1']

bits_csv = ('/home/resperanca/Tuberculosis_Tese/Source/Models/SHAP/shap_top_ecfp_bits.csv')  # ex.: "shap_top_ecfp_bits.csv"
topN = 20       



if bits_csv:
    df_bits = pd.read_csv(bits_csv)
    important_bits = df_bits["bit"].astype(int).tolist()[:topN]
    shap_means = df_bits.set_index("bit")["shap_value_mean"]
else:
    shap_means = pd.Series(dtype=float)  # vazio (sem pesos)

important_bits = list(map(int, important_bits))
important_bits_sorted = sorted(important_bits)  # só para ordem fixa nas colunas

def ecfp_bits_presence(smi, bits, radius=2, nBits=1024):
    """Retorna dict com presença 0/1 por bit e lista dos bits presentes."""
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return None, {}
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
    presence = {b: int(fp[b]) for b in bits}
    return mol, presence

rows = []
for smi in smiles_list_1:
    mol, presence = ecfp_bits_presence(smi, important_bits_sorted)
    if mol is None:
        rows.append({
            "smiles": smi,
            "valid_smiles": 0,
            "n_bits_present": np.nan,
            "bits_present": "",
            "weighted_score": np.nan,
            **{f"bit_{b}": np.nan for b in important_bits_sorted},
        })
        continue

   
    n_present = sum(presence.values())
    bits_present_list = [b for b,v in presence.items() if v==1]

    
    if not shap_means.empty:
        weighted = float(sum(shap_means.get(b, 0.0) for b in bits_present_list))
    else:
        weighted = np.nan

    base = {
        "smiles": smi,
        "valid_smiles": 1,
        "n_bits_present": n_present,
        "bits_present": ",".join(map(str, bits_present_list)),
        "weighted_score": weighted,
    }
    # colunas 0/1 por bit
    base.update({f"bit_{b}": presence[b] for b in important_bits_sorted})
    rows.append(base)

df = pd.DataFrame(rows)


if "weighted_score" in df.columns and df["weighted_score"].notna().any():
    df = df.sort_values(["weighted_score","n_bits_present"], ascending=False, na_position="last")
else:
    df = df.sort_values("n_bits_present", ascending=False)

print(df[["smiles","valid_smiles","n_bits_present","bits_present","weighted_score"]].to_string(index=False))
# Se quiseres exportar:
df.to_csv("bits_presence_report.csv", index=False)


                                                  smiles  valid_smiles  n_bits_present bits_present  weighted_score
CCCS(=O)(=O)C1CCN(c2ccn3cc(-c4cc(Cl)c(OC)cc4OC)nc3n2)CC1             1               2        49,61        0.215907


In [ ]:
import pandas as pd

dataset_path = "/home/resperanca/Tuberculosis_Tese/Source/code/SHAP_results/bits_presence_report_14.csv" 
df = pd.read_csv(dataset_path)
df

,smiles,valid_smiles,n_bits_present,bits_present,weighted_score,bit_0,bit_6,bit_39,bit_49,bit_58,...,bit_100,bit_106,bit_123,bit_137,bit_156,bit_160,bit_178,bit_218,bit_223,bit_472
0,NC(=O)c1cc(-c2ccc(Cl)c(Cl)c2)cc2c1[nH]c1ccc(C(...,1,1,223.0,0.062914,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,Cn1cnc(S(=O)(=O)Nc2cccc(-c3ccc(C(=O)c4cc(F)c(F...,1,1,160.0,0.054373,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,Nc1cncc(C2=NN(C(=O)c3ccc(-c4ccccn4)s3)C(c3cccc...,1,1,160.0,0.054373,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,CNC(=O)C(Cc1ccccc1)NC(=O)C(CC(C)C)C1(C(=O)NO)C...,1,0,NaN,0.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CNC(=O)C(Cc1ccccc1)NC(=O)C(CC(C)C)C1(C(=O)NO)C...,1,0,NaN,0.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,COc1cc(C#N)cc(-c2csc(C34CN(c5ncc(F)cn5)CC3C(=O...,1,0,NaN,0.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Cc1c(CC(=O)O)cc2ccc(F)cc2c1-c1ccc(S(=O)(=O)N2C...,1,0,NaN,0.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,O=C(NCC1CN(c2ccc(N3CCCC(O)C3)c(F)c2)C(=O)O1)c1...,1,0,NaN,0.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,CCc1coc(NC(=O)c2cc3c(nc2OC)nc(C(O)(c2ccccc2)c2...,1,0,NaN,0.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,CNC(=O)C1(c2cccc(OCc3cc(C)nc4ccccc34)c2)CC1C(=...,1,0,NaN,0.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
import pandas as pd

dataset_path = "/home/resperanca/Tuberculosis_Tese/Source/code/SHAP_results/bits_presence_report_1.csv" 
df = pd.read_csv(dataset_path)
df

,smiles,valid_smiles,n_bits_present,bits_present,weighted_score,bit_0,bit_6,bit_39,bit_49,bit_58,...,bit_100,bit_106,bit_123,bit_137,bit_156,bit_160,bit_178,bit_218,bit_223,bit_472
0,CCCS(=O)(=O)C1CCN(c2ccn3cc(-c4cc(Cl)c(OC)cc4OC...,1,2,"49,61",0.215907,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Generate representations of the molecules in question with the present bits highlighted

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from IPython.display import display
from PIL import Image as PILImage
import io, os, math, random


smiles_list = [
    "NC(=O)c1cc(-c2ccc(Cl)c(Cl)c2)cc2c1[nH]c1ccc(C(=O)N3CCOC(CO)C3)cc12",
    "Cn1cnc(S(=O)(=O)Nc2cccc(-c3ccc(C(=O)c4cc(F)c(F)c(O)c4F)s3)c2)c1",
    "Nc1cncc(C2=NN(C(=O)c3ccc(-c4ccccn4)s3)C(c3ccccc3O)C2)c1",
    "CCCS(=O)(=O)C1CCN(c2ccn3cc(-c4cc(Cl)c(OC)cc4OC)nc3n2)CC1",
]
top_bits = [49,472,178,123,100,0,137,223,39,88,156,91,106,72,71,160,218,61,6,58]

save_dir = "bits_mol_e_fragmento"   
mols_per_row = 3                    
subimg_size = (240, 240)            
radius = 2
nBits = 1024
max_smiles_len = 40

def _short(s, k=40):
    return s if len(s) <= k else s[:k-3] + "..."

def get_bit_envs(mol, bit, radius=2, nBits=1024):
    """
    Para um 'bit' Morgan, devolve lista de ocorrências na molécula:
    cada ocorrência contém 'atoms' (list), 'bonds' (list), 'center' (int), 'rad' (int).
    """
    bit_info = {}
    _ = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits, bitInfo=bit_info)
    if bit not in bit_info:
        return []

    envs = []
    for atom_idx, rad in bit_info[bit]:
        bond_ids = Chem.FindAtomEnvironmentOfRadiusN(mol, rad, atom_idx)
        atom_ids = set()
        for b in bond_ids:
            bond = mol.GetBondWithIdx(b)
            atom_ids.add(bond.GetBeginAtomIdx())
            atom_ids.add(bond.GetEndAtomIdx())
        if not bond_ids:  
            atom_ids.add(atom_idx)
        envs.append({
            "atoms": sorted(atom_ids),
            "bonds": list(bond_ids),
            "center": atom_idx,
            "rad": rad,
        })
    return envs

def submol_from_env(mol, env):
    """
    Constrói o submol (fragmento) a partir de uma ocorrência (env).
    Usa PathToSubmol; quando não há ligações (rad=0), força inclusão de átomos.
    """
    bonds = env["bonds"]
    atoms = env["atoms"]
    if bonds:
        sub = Chem.PathToSubmol(mol, bonds)
    else:
    
        try:
            sub = Chem.PathToSubmol(mol, bondIndices=[], atomIndices=atoms)
        except TypeError:
            
            em = Chem.EditableMol(Chem.Mol())
            map_old_to_new = {}
            for aidx in atoms:
                a = mol.GetAtomWithIdx(aidx)
                na = Chem.Atom(a.GetAtomicNum())
                new_idx = em.AddAtom(na)
                map_old_to_new[aidx] = new_idx
            sub = em.GetMol()
    
    try:
        if sub.GetNumConformers() == 0:
            AllChem.Compute2DCoords(sub)
    except Exception:
        AllChem.Compute2DCoords(sub)
    return sub

def mol_to_image(mol, legend, highlightAtoms=None, highlightBonds=None, size=(240,240), color=(0.98,0.25,0.25)):
    """
    Desenha uma única imagem da molécula inteira com destaque (quando fornecido).
    """
    highlightAtoms = highlightAtoms or []
    highlightBonds = highlightBonds or []
    try:
        img = Draw.MolToImage(
            mol, size=size, legend=legend,
            highlightAtoms=highlightAtoms,
            highlightBonds=highlightBonds,
            highlightColor=color
        )
    except TypeError:
        img = Draw.MolToImage(
            mol, size=size, legend=legend,
            highlightAtoms=highlightAtoms,
            highlightBonds=highlightBonds
        )
    return img  

def hpair(left_img, right_img, tile_size=(240,240), gap=8, bg=(255,255,255)):
    """
    Cria um painel horizontal (par) juntando duas imagens do mesmo tamanho, com um espaçamento central.
    """
    w, h = tile_size
    canvas = PILImage.new("RGB", (w*2 + gap, h), bg)
    if left_img.size != tile_size:
        left_img = left_img.resize(tile_size, resample=PILImage.BICUBIC)
    if right_img.size != tile_size:
        right_img = right_img.resize(tile_size, resample=PILImage.BICUBIC)
    canvas.paste(left_img, (0, 0))
    canvas.paste(right_img, (w + gap, 0))
    return canvas

def make_grid(panels, molsPerRow=3, tile_size=(240,240), gap=8, bg=(255,255,255)):
    """
    Monta uma grade com 'pares' (cada painel é já uma imagem larga: 2*tile_width+gap).
    """
    if not panels:
        raise ValueError("Lista de painéis vazia.")
    single_w, single_h = tile_size
    pair_w = single_w*2 + gap
    pair_h = single_h
    n = len(panels)
    rows = math.ceil(n / molsPerRow)
    grid = PILImage.new("RGB", (molsPerRow * pair_w, rows * pair_h), bg)
    for idx, im in enumerate(panels):
        if im.size != (pair_w, pair_h):
            im = im.resize((pair_w, pair_h), resample=PILImage.BICUBIC)
        r = idx // molsPerRow
        c = idx % molsPerRow
        grid.paste(im, (c * pair_w, r * pair_h))
    return grid


# MAIN

def show_bits_mol_and_fragment(smiles_list, bits, save_dir=None,
                               molsPerRow=3, subImgSize=(240,240),
                               radius=2, nBits=1024, max_smiles_len=40, pair_gap=8):
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)

    rng = random.Random(42)  
    bit_colors = {}
    def color_for(bit):
        if bit not in bit_colors:
            bit_colors[bit] = (0.9*rng.random(), 0.5+0.5*rng.random(), 0.4+0.6*rng.random())
        return bit_colors[bit]

    for i, smi in enumerate(smiles_list):
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            print(f"[Mol {i}] SMILES inválido: {smi}")
            continue

        if mol.GetNumConformers() == 0:
            AllChem.Compute2DCoords(mol)

        pairs = []
        for b in bits:
            envs = get_bit_envs(mol, b, radius=radius, nBits=nBits)
            col = color_for(b)
            for k, env in enumerate(envs, 1):
               
                legend_left = f"Mol {i} | Bit {b} (oc {k})"
                mol_img = mol_to_image(
                    mol, legend_left,
                    highlightAtoms=env["atoms"],
                    highlightBonds=env["bonds"],
                    size=subImgSize,
                    color=col
                )
                
                frag = submol_from_env(mol, env)
                legend_right = f"Fragmento bit {b} (oc {k})"
                try:
                    frag_img = Draw.MolToImage(
                        frag, size=subImgSize, legend=legend_right,
                        highlightAtoms=list(range(frag.GetNumAtoms())),
                        highlightColor=col
                    )
                except TypeError:
                    frag_img = Draw.MolToImage(frag, size=subImgSize, legend=legend_right)

                
                pair = hpair(mol_img, frag_img, tile_size=subImgSize, gap=pair_gap)
                pairs.append(pair)

            if not pairs:
                print(f"[Mol {i}] Nenhuma ocorrência dos bits {bits}.")
                continue

            
            grid = make_grid(pairs, molsPerRow=molsPerRow, tile_size=subImgSize, gap=pair_gap)
            display(grid)

            if save_dir:
                out_path = os.path.join(save_dir, f"mol_{i}_mol_e_fragmentos.png")
                grid.save(out_path)
                print(f"[Mol {i}] imagem salva em: {out_path}")



# RUN

show_bits_mol_and_fragment(
    smiles_list, top_bits,
    save_dir=save_dir,
    molsPerRow=mols_per_row,      
    subImgSize=subimg_size,
    radius=radius, nBits=nBits,
    max_smiles_len=max_smiles_len,
    pair_gap=8
)


## Final analisys

In [ ]:
df = pd.read_csv("/home/resperanca/Tuberculosis_Tese/Data/Tuberculosis_Data/tb_final3_READY.csv")
SEQ = "MTGLLDGKRILVSGIITDSSIAFHIARVAQEQGAQLVLTGFDRLRLIQRITDRLPAKAPLLELDVQNEEHLASLAGRVTEAIGAGNKLDGVVHSIGFMPQTGMGINPFFDAPYADVSKGIHISAYSYASMAKALLPIMNPGGSIVGMDFDPSRAMPAYNWMTVAKSALESVNRFVAREAGKYGVRSNLVAAGPIRTLAMSAIVGGALGEEAGAQIQLLEEGWDQRAPIGWNMKDATPVAKTVCALLSDWLPATTGDIIYADGGAHTQLL"  
hits = df[df["Target Sequence"] == SEQ]
print(hits)  

In [ ]:
import pandas as pd

df = pd.read_csv("/home/resperanca/Tuberculosis_Tese/Data/DTI_Data_2_step/Papyrus_final.csv")
smi = "CCCS(=O)(=O)C1CCN(c2ccn3cc(-c4cc(Cl)c(OC)cc4OC)nc3n2)CC1"  
hits = df[df["SMILES"] == smi]
print(hits)   

In [ ]:
#top 10
"NC(=O)c1cc(-c2ccc(Cl)c(Cl)c2)cc2c1[nH]c1ccc(C(=O)N3CCOC(CO)C3)cc12"
'CHEMBL3646837'
'https://patentimages.storage.googleapis.com/de/4c/72/4fcdff4b086a7f/US8815840.pdf'
'MTEIATTSGARSVGLLSVGAYRPERVVTNDEICQHIDSSDEWIYTRTGIKTRRFAADDESAASMATEACRRALSNAGLSAADIDGVIVTTNTHFLQTPPAAPMVAASLGAKGILGFDLSAGCAGFGYALGAAADMIRGGGAATMLVVGTEKLSPTIDMYDRGNCFIFADGAAAVVVGETPFQGIGPTVAGSDGEQADAIRQDIDWITFAQNPSGPRPFVRLEGPAVFRWAAFKMGDVGRRAMDAAGVRPDQIDVFVPHQANSRINELLVKNLQLRPDAVVANDIEHTGNTSAASIPLAMAELLTTGAAKPGDLALLIGYGAGLSYAAQVVRMPKG'
'P9WNG3'
'0.990252'

"Cn1cnc(S(=O)(=O)Nc2cccc(-c3ccc(C(=O)c4cc(F)c(F)c(O)c4F)s3)c2)c1"
'CHEMBL4519484'
'10.1021/acs.jmedchem.9b00932'
'MLQQIRGPADLQHLSQAQLRELAAEIREFLIHKVAATGGHLGPNLGVVELTLALHRVFDSPHDPIIFDTGHQAYVHKMLTGRSQDFATLRKKGGLSGYPSRAESEHDWVESSHASAALSYADGLAKAFELTGHRNRHVVAVVGDGALTGGMCWEALNNIAASRRPVIIVVNDNGRSYAPTIGGVADHLATLRLQPAYEQALETGRDLVRAVPLVGGLWFRFLHSVKAGIKDSLSPQLLFTDLGLKYVGPVDGHDERAVEVALRSARRFGAPVIVHVVTRKGMGYPPAEADQAEQMHSTVPIDPATGQATKVAGPGWTATFSDALIGYAQKRRDIVAITAAMPGPTGLTAFGQRFPDRLFDVGIAEQHAMTSAAGLAMGGLHPVVAIYSTFLNRAFDQIMMDVALHKLPVTMVLDRAGITGSDGASHNGMWDLSMLGIVPGIRVAAPRDATRLREELGEALDVDDGPTALRFPKGDVGEDISALERRGGVDVLAAPADGLNHDVLLVAIGAFAPMALAVAKRLHNQGIGVTVIDPRWVLPVSDGVRELAVQHKLLVTLEDNGVNGGAGSAVSAALRRAEIDVPCRDVGLPQEFYEHASRSEVLADLGLTDQDVARRITGWVAALGTGVCASDAIPEHLD'
'P9WNS3'
'0.991699'

"Nc1cncc(C2=NN(C(=O)c3ccc(-c4ccccn4)s3)C(c3ccccc3O)C2)c1"
'CHEMBL1778675'
'10.1016/j.bmcl.2010.06.113'
'MTIPAFHPGELNVYSAPGDVADVSRALRLTGRRVMLVPTMGALHEGHLALVRAAKRVPGSVVVVSIFVNPMQFGAGEDLDAYPRTPDDDLAQLRAEGVEIAFTPTTAAMYPDGLRTTVQPGPLAAELEGGPRPTHFAGVLTVVLKLLQIVRPDRVFFGEKDYQQLVLIRQLVADFNLDVAVVGVPTVREADGLAMSSRNRYLDPAQRAAAVALSAALTAAAHAATAGAQAALDAARAVLDAAPGVAVDYLELRDIGLGPMPLNGSGRLLVAARLGTTRLLDNIAIEIGTFAGTDRPDGYRAILESHWRN'
'P9WIL5'
'0.994189'

#iso

"CCCS(=O)(=O)C1CCN(c2ccn3cc(-c4cc(Cl)c(OC)cc4OC)nc3n2)CC1"
'CHEMBL4566916'
'http://patentimages.storage.googleapis.com/7e/c3/3e/e32a4fd52861af/US10508109.pdf'
'MTGLLDGKRILVSGIITDSSIAFHIARVAQEQGAQLVLTGFDRLRLIQRITDRLPAKAPLLELDVQNEEHLASLAGRVTEAIGAGNKLDGVVHSIGFMPQTGMGINPFFDAPYADVSKGIHISAYSYASMAKALLPIMNPGGSIVGMDFDPSRAMPAYNWMTVAKSALESVNRFVAREAGKYGVRSNLVAAGPIRTLAMSAIVGGALGEEAGAQIQLLEEGWDQRAPIGWNMKDATPVAKTVCALLSDWLPATTGDIIYADGGAHTQLL'
'P9WGR1'
'0.995174'

